### Import

In [1]:
import numpy as np
import sklearn
import pandas as pd
import matplotlib
import seaborn as sns

# 각 라이브러리 버전 출력
print("Numpy version:", np.__version__)
print("Scikit-learn version:", sklearn.__version__)
print("Pandas version:", pd.__version__)
print("Matplotlib version:", matplotlib.__version__)
print("Seaborn version:", sns.__version__)

Numpy version: 2.0.1
Scikit-learn version: 1.6.1
Pandas version: 2.2.3
Matplotlib version: 3.10.0
Seaborn version: 0.13.2


In [2]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import  OrdinalEncoder
from sklearn.ensemble import ExtraTreesClassifier

### Data Load

In [3]:
import sys
import os
import pandas as pd

# 현재 작업 디렉토리 경로를 가져와 shared codes 폴더의 위치를 sys.path에 추가합니다.
# sys.path에 추가된 경로에 있는 py 폴더는 임포트할 수 있다.
current_dir = os.getcwd()
shared_codes_dir = os.path.join(current_dir, '../shared codes')
sys.path.append(shared_codes_dir)


# cover_nan 모듈을 임포트
from cover_nan_0215_dahun import missing_value_removal_function

# 원본 train 데이터 로드
train = pd.read_csv("../shared codes/data/train.csv")
test = pd.read_csv("../shared codes/data/test.csv")

# missing_value_removal_function 사용
train_young, train_middle, train_old, train_unknown = missing_value_removal_function(train)
test_young, test_middle, test_old, test_unknown = missing_value_removal_function(test)

✅ '대리모 여부' 결측값을 최빈값 (0.0) 으로 대체 완료!
✅ 컬럼 삭제 완료: ['PGD 시술 여부', 'PGS 시술 여부', '난자 해동 경과일', '배아 해동 경과일']
✅ '난자 채취 경과일' 결측값을 중앙값 (0.0) 으로 대체 완료!
✅ '난자 혼합 경과일' 결측값을 중앙값 (0.0) 으로 대체 완료!
✅ '배아 이식 경과일' 결측값을 중앙값 (3.0) 으로 대체 완료!
50    50
51    51
52    52
53    53
54    54
55    55
56    56
57    57
58    58
59    59
60    60
61    61
62    62
63    63
64    64
65    65
66    66
67    67
68    68
69    69
70    70
71    71
72    72
73    73
74    74
75    75
76    76
77    77
78    78
79    79
80    80
81    81
82    82
83    83
84    84
85    85
86    86
87    87
88    88
89    89
90    90
91    91
92    92
93    93
94    94
95    95
96    96
97    97
98    98
99    99
Name: index, dtype: int64
✅ '대리모 여부' 결측값을 최빈값 (0.0) 으로 대체 완료!
✅ 컬럼 삭제 완료: ['PGD 시술 여부', 'PGS 시술 여부', '난자 해동 경과일', '배아 해동 경과일']
✅ '난자 채취 경과일' 결측값을 중앙값 (0.0) 으로 대체 완료!
✅ '난자 혼합 경과일' 결측값을 중앙값 (0.0) 으로 대체 완료!
✅ '배아 이식 경과일' 결측값을 중앙값 (3.0) 으로 대체 완료!
50    50
51    51
52    52
53    53
54    54
55    55
56    56
57    57
58    58
59    5

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler

import optuna
from sklearn.ensemble import VotingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, cross_val_score
import numpy as np

def data_preprocessing(train, test):
    index_train = train['index']
    X = train.drop(['임신 성공 여부', 'index'], axis=1)
    y = train['임신 성공 여부']

    index_test = test['index']
    test = test.drop('index', axis=1)

    #Data Pre-processing
    # Categorical(범주형) 칼럼 찾기
    categorical_columns = X.select_dtypes(include=['object', 'category']).columns.tolist()




    ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

    X_train_encoded = X.copy()
    X_train_encoded[categorical_columns] = ordinal_encoder.fit_transform(X[categorical_columns])

    X_test_encoded = test.copy()
    X_test_encoded[categorical_columns] = ordinal_encoder.transform(test[categorical_columns])

    columns_to_drop = [
            "남성 주 불임 원인",
            "남성 부 불임 원인",
            "불임 원인 - 정자 농도",
            "불임 원인 - 정자 면역학적 요인",
            "불임 원인 - 정자 운동성",
            "불임 원인 - 정자 형태",
            '배란 유도 유형'
    ]
    X_train_encoded = X_train_encoded.drop(columns = columns_to_drop)    
    X_test_encoded = X_test_encoded.drop(columns = columns_to_drop)
    



    # 데이터 정규화 (X_train_encoded & X_test_encoded)
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train_encoded)
    X_test_scaled = scaler.transform(X_test_encoded)  # 동일한 스케일 적용

    # DataFrame 변환 (Feature 이름 유지)
    feature_names = [f"Feature_{i}" for i in range(X_train_scaled.shape[1])]
    X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=feature_names)
    X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=feature_names)

    # 상관 행렬 계산
    correlation_matrix_train = X_train_scaled_df.corr()

    # 다중 공선성이 높은 칼럼 찾기 (절대 상관 계수가 0.8 이상)
    threshold = 0.8
    high_corr_features = set()

    for i in range(len(feature_names)):
        for j in range(i + 1, len(feature_names)):
            if abs(correlation_matrix_train.iloc[i, j]) > threshold:
                high_corr_features.add(feature_names[j])  # 공선성이 높은 컬럼 추가

    # 다중 공선성이 높은 컬럼 제거
    X_train_encoded = X_train_scaled_df.drop(columns=high_corr_features, errors='ignore')
    X_test_encoded = X_test_scaled_df.drop(columns=high_corr_features, errors='ignore')

    X_train_encoded['index'] = index_train
    X_test_encoded['index'] = index_test

    return X_train_encoded, X_test_encoded, y


/opt/anaconda3/envs/lg_aimers/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
def optimize_models_and_get_best_params(X_train_encoded, y):

    def optimize_xgboost(trial):
        params = {
            "n_estimators": trial.suggest_int("n_estimators", 100, 500),
            "max_depth": trial.suggest_int("max_depth", 3, 12),
            "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
            "subsample": trial.suggest_float("subsample", 0.6, 1.0),
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
            "gamma": trial.suggest_float("gamma", 0, 5)
        }
        model = XGBClassifier(**params, random_state=42, eval_metric="logloss")
        scores = cross_val_score(model, X_train_encoded, y, cv=5, scoring="roc_auc")
        return np.mean(scores)


    def optimize_lightgbm(trial):
        params = {
            "n_estimators": trial.suggest_int("n_estimators", 100, 500),
            "max_depth": trial.suggest_int("max_depth", 3, 12),
            "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
            "num_leaves": trial.suggest_int("num_leaves", 10, 50),
            "subsample": trial.suggest_float("subsample", 0.6, 1.0)
        }
        model = LGBMClassifier(**params, random_state=42)
        scores = cross_val_score(model, X_train_encoded, y, cv=5, scoring="roc_auc")  
        return np.mean(scores)

    def optimize_random_forest(trial):
        params = {
            "n_estimators": trial.suggest_int("n_estimators", 100, 300),
            "max_depth": trial.suggest_int("max_depth", 3, 12),
            "min_samples_split": trial.suggest_int("min_samples_split", 3, 10),
            "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 4)
        }
        model = RandomForestClassifier(**params, random_state=42)
        scores = cross_val_score(model, X_train_encoded, y, cv=5, scoring="roc_auc")  
        return np.mean(scores)

    # Optuna 스터디 실행
    print("Optimizing XGBoost...")
    xgb_study = optuna.create_study(direction="maximize")
    xgb_study.optimize(optimize_xgboost, n_trials=5)
    best_xgb_params = xgb_study.best_params

    print("Optimizing LightGBM...")
    lgbm_study = optuna.create_study(direction="maximize")
    lgbm_study.optimize(optimize_lightgbm, n_trials=5)
    best_lgbm_params = lgbm_study.best_params

    print("Optimizing RandomForest...")
    rf_study = optuna.create_study(direction="maximize")
    rf_study.optimize(optimize_random_forest, n_trials=2)
    best_rf_params = rf_study.best_params

    params = [best_xgb_params, best_lgbm_params, best_rf_params]
    
    return params


In [6]:
def train_ensemble_model(X_train_encoded, y, X_test_encoded, params):

    
    # best params
    best_xgb_params, best_lgbm_params, best_rf_params = params

    # 최적화된 모델 생성
    xgb_model = XGBClassifier(**best_xgb_params, random_state=42, use_label_encoder=False, eval_metric="logloss")
    lgbm_model = LGBMClassifier(**best_lgbm_params, random_state=42)
    rf_model = RandomForestClassifier(**best_rf_params, random_state=42)

    # Soft Voting 앙상블
    ensemble_model = VotingClassifier(
        estimators=[
            ("xgb", xgb_model),
            ("lgbm", lgbm_model),
            ("rf", rf_model)
        ],
        voting="soft"
    )

    # 전체 데이터로 학습
    ensemble_model.fit(X_train_encoded, y)

    pred_proba = ensemble_model.predict_proba(X_test_encoded)[:, 1]

    submission = pd.DataFrame()
    submission['probability'] = pred_proba
    submission['index'] = X_test_encoded['index']

    return submission

In [ ]:
X_train_encoded_young, X_test_encoded_young, y_young = data_preprocessing(train_young, test_young)
X_train_encoded_middle, X_test_encoded_middle, y_middle = data_preprocessing(train_middle, test_middle)
X_train_encoded_old, X_test_encoded_old, y_old = data_preprocessing(train_old, test_old)

params_young = optimize_models_and_get_best_params(X_train_encoded_young, y_young)
params_middle = optimize_models_and_get_best_params(X_train_encoded_middle, y_middle)
params_old = optimize_models_and_get_best_params(X_train_encoded_old, y_old)

submission_young = train_ensemble_model(X_train_encoded_young, y_young, X_test_encoded_young, params_young)
submission_middle = train_ensemble_model(X_train_encoded_middle, y_middle, X_test_encoded_middle, params_middle)
submission_old = train_ensemble_model(X_train_encoded_old, y_old, X_test_encoded_old, params_old)


In [28]:
# test_unknown의 데이터 개수만큼 0으로 채워진 submission_unknown 생성
submission_unknown = pd.DataFrame()
submission_unknown['probability'] = [0] * len(test_unknown)
submission_unknown['index'] = test_unknown.index

submission = pd.concat([submission_young, submission_middle, submission_old, submission_unknown])

submission = submission.sort_values('index')
# submission = submission.drop(columns=['index'])

home_dir = '/Users/downy/Documents/2025 LG aimers/DKU-LG-Capstone-6'
sample_submission = pd.read_csv(home_dir + '/shared codes/data/sample_submission.csv')

submission['ID'] = sample_submission['ID']

# print(sample_submission[60000:60050])

submission = submission[['ID', 'probability']]

submission.to_csv("submission.csv", index=False)



In [24]:
submission_unknown['index'] = test_unknown.index
submission_unknown

,probability,index
0,0,749
1,0,756
2,0,810
3,0,2017
4,0,2213
...,...,...
230,0,89178
231,0,89298
232,0,89484
233,0,89699


In [13]:
submission

,probability,index
0,0.020112,0.0
1,0.463508,1.0
2,0.117618,2.0
3,0.058717,3.0
4,0.406301,4.0
...,...,...
230,0.000000,NaN
231,0.000000,NaN
232,0.000000,NaN
233,0.000000,NaN


In [ ]:
submission['ID'] = sample_submission['ID']

# print(sample_submission[60000:60050])

submission = submission[['ID', 'probability']]

submission.to_csv("submission.csv", index=False)

In [9]:
submission = submission[['ID', 'probability']]

submission.to_csv("submission.csv", index=False)

In [10]:
home_dir = '/Users/downy/Documents/2025 LG aimers/DKU-LG-Capstone-6'
sample_submission = pd.read_csv(home_dir + '/shared codes/data/sample_submission.csv')

print(sample_submission[60000:60050])

               ID  probability
60000  TEST_60000          0.0
60001  TEST_60001          0.0
60002  TEST_60002          0.0
60003  TEST_60003          0.0
60004  TEST_60004          0.0
60005  TEST_60005          0.0
60006  TEST_60006          0.0
60007  TEST_60007          0.0
60008  TEST_60008          0.0
60009  TEST_60009          0.0
60010  TEST_60010          0.0
60011  TEST_60011          0.0
60012  TEST_60012          0.0
60013  TEST_60013          0.0
60014  TEST_60014          0.0
60015  TEST_60015          0.0
60016  TEST_60016          0.0
60017  TEST_60017          0.0
60018  TEST_60018          0.0
60019  TEST_60019          0.0
60020  TEST_60020          0.0
60021  TEST_60021          0.0
60022  TEST_60022          0.0
60023  TEST_60023          0.0
60024  TEST_60024          0.0
60025  TEST_60025          0.0
60026  TEST_60026          0.0
60027  TEST_60027          0.0
60028  TEST_60028          0.0
60029  TEST_60029          0.0
60030  TEST_60030          0.0
60031  T